In [0]:
import tensorflow as tf
import numpy as np
from music21 import *
import os
import json
import numpy as np
import pickle
from random import shuffle
import zipfile
import gensim
import random
from datetime import datetime
import matplotlib.pyplot as plt
import shutil

length_of_1 = 100
from google.colab import drive
drive.mount('/content/gdrive')
file_string = '/content/gdrive/My Drive/LSTMGANV2'

In [0]:

def next_batch(lis):
    counter = 0
    string = ""
    dataset = {}
    for item in lis:
        string = str(item)
        pickle_file = open(file_string + "/embed_data/" + string + ".pickle","rb")
        lis = pickle.load(pickle_file)
        pickle_file.close()
        ray = np.array(lis)
        ray = np.reshape(ray, [1,ray.shape[0]])
        dataset[item] = ray
        print(dataset[item])
        counter = counter + 1
        print(counter)
    return (dataset)
dataset = next_batch(range(200))

In [0]:

input_music = tf.keras.Input([length_of_1])

embedded = tf.keras.layers.Embedding(89,256)(input_music)

LSTM1 = tf.keras.layers.GRU(1024, activation = tf.keras.activations.relu, return_sequences = True, recurrent_initializer='glorot_uniform')(embedded)
#LSTM2 = tf.keras.layers.LSTM(512, activation = tf.keras.activations.relu, return_sequences = True)(LSTM1)
dense2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(89, activation = tf.keras.activations.softmax))(LSTM1)
#dense2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(89, activation = tf.keras.activations.softmax))(dense1)

model = tf.keras.Model(input_music, dense2)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate = .001)

model.compile(optimizer = optimizer_adam , loss='sparse_categorical_crossentropy')


print(model.summary())

In [0]:
def get_data(lis, length, dataset):
  counter = 0
  length = length + 1
  for item in lis:
    ray = dataset[item]
    index = random.randint(0,ray.shape[1]-1-length)
    ray = ray[:, index:index+length]
    
    if counter == 0:
      final = ray
    else:
      final =  np.concatenate((final, ray),0)
    counter += 1
  return final

#data = get_data([121,122,123,23,5,5,6,3,2,2], 100, dataset)

In [0]:
#num_total_data = 15509
num_total_data = 15509
batch_size = 100
epochs = 200

In [0]:
loss_history = []
for e in range(epochs):
    num_batches = num_total_data // batch_size
    data_list = list(range(num_total_data))
    random.shuffle(data_list)
    for i in range(num_batches):
        print("getting data")
        start_data = get_data(data_list[i*batch_size:i*batch_size + batch_size], length_of_1, dataset)
        print("got data")
        print(start_data.shape)
        print("train start")

        loss = model.train_on_batch(start_data[:,:-1], start_data[:,1:])
        print("TRAIN END")

        loss_history.append(loss)
        print("LOSS IS {}".format(loss))
    
        if i % 50 == 0 and i != 0:
            print("5 batches completed")
            singular_start = start_data[0,:-1]
            print(singular_start)
            for iteration in range(100):
              next_step = model.predict(singular_start[-100:].reshape(1,100))
              predictions = next_step[0,-1,:]
              
              #predictions = predictions / temperature
              predicted_id = np.random.choice(range(89), p = predictions)
              singular_start = np.concatenate((singular_start, predicted_id.reshape(1)), axis = 0)
            
            
            print(singular_start[-100:])
            with open(file_string + "/music_samples/{}.pickle".format(i), "wb+") as file:
                pickle.dump(singular_start, file)

        model.save(file_string + "/models/BASICLSTM.h5")
                              
                        

In [0]:
loss_history = []
model = tf.keras.models.load_model(file_string + "/models/BASICLSTM.h5")
for e in range(epochs):
    num_batches = num_total_data // batch_size
    data_list = list(range(num_total_data))
    random.shuffle(data_list)
    for i in range(num_batches):
        print("getting data")
        start_data = next_batch(data_list[i*batch_size:i*batch_size + batch_size], length_of_1 + 1)
        print("got data")
        print(start_data.shape)
        loss = model.train_on_batch(start_data[:,:-1,:], start_data[:,-1,:])
        

        loss_history.append(loss)
        print("LOSS IS {}".format(loss))
    
        if i % 10 == 0:
            print("10 batches completed")
            singular_start = start_data[0,:-1,:]
            for iteration in range(100):
              next_step = model.predict(singular_start[-100:, :].reshape(1,100,89))
              singular_start = np.concatenate((singular_start, next_step), axis = 0)
            
            results = np.argmax(singular_start, axis = 1)
            print(results)
            with open(file_string + "/music_samples/{}.pickle".format(i), "wb+") as file:
                pickle.dump(results, file)

        model.save(file_string + "/models/BASICLSTM.h5")
                              
                        

In [0]:
model_load = tf.keras.models.load_model(file_string + "/models/BASICLSTM.h5")
print(model_load.summary())
test_data = dataset[3]
test_data = test_data[0,:100]
print(test_data)
temperature = 1

for iteration in range(10000):
  next_step = model_load.predict(test_data[-100:].reshape(1,100))
  #print(next_step.shape)
  predictions = next_step[0,-1,:]
  predictions = predictions / temperature
  int_test = random.randint(0,10)
  if int_test <4:
    predicted_id = np.random.choice(range(89), p = predictions)
  else:
    predicted_id = np.argmax(predictions)
  #print(predicted_id)
  print(iteration)
  test_data = np.concatenate((test_data, predicted_id.reshape(1)), axis = 0)
  with open(file_string + "/music_samples/temp_music.pickle", "wb+") as file:
    pickle.dump(test_data, file)


In [0]:
import sys
np.set_printoptions(threshold=sys.maxsize)
print(test_data[:-600])

In [0]:
lis = list(test_data[:-600])
print(lis)


In [0]:
split_list = []
temp_list = []
for item in lis:
  if item == 88:
    split_list.append(temp_list)
    temp_list = []
  else:
    temp_list.append(item)
print(split_list)
    



In [0]:
array = np.zeros((len(split_list),88))
for x in range(len(split_list)):
  array[x,split_list[x]] = 1
print(array)

In [0]:
import pickle
from music21 import *
def reconstruct(ray):
  stream1 = stream.Stream()
  print(ray.shape)
  for x in range(ray.shape[0]):
    for y in range(ray.shape[1]):
      if (ray[x,y] == 1) and (ray[x-1,y] == 0):
        w = 0
        while ray[x+w, y] == 1:
          if (x+w+1<ray.shape[0]):
            w += 1
          else:
            w += 1
            break
        w += 2
        print(w)
        note1 = note.Note(y+40) #check this
        note1.quarterLength = w/24
        stream1.append(note1)
        note1.offset = x/24 #is this the right way?
  return(stream1)
midi = reconstruct(array)
midi.show('text')

In [0]:
fp = midi.write('midi', fp=(file_string + "/music_samples/fourth_test_midi.mid"))